In [4]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoModelForMaskedLM

config = AutoConfig.from_pretrained("bert-base-cased")
masked_model = AutoModelForMaskedLM.from_config(config)
model = AutoModelForSequenceClassification.from_config(config)

In [12]:
model.from_pretrained('./test_trainer/runs/Mar21_00-02-43_SaiMiKiTO')

AttributeError: 'BertForSequenceClassification' object has no attribute 'load'